In [1]:
import time
from matplotlib import pyplot as plt
import csv
import os
import datetime
import serial
import serial.tools.list_ports
from matplotlib.ticker import NullFormatter
import threading
import sys


#########################################################################################################
def seleccionar_puerto():
    puertos_disponibles = [puerto.device for puerto in serial.tools.list_ports.comports()]

    if not puertos_disponibles:
        print("No se encontraron puertos seriales disponibles.")
        return None
    print("------------------------------------------------")
    print("SELECCIONE PUERTO SERIAL:")
    for indice, puerto in enumerate(puertos_disponibles, start=1):
        print(f"{indice}. {puerto}")

    seleccion = 0
    while True:
        try:
            seleccion = int(input("Seleccione el número del puerto a usar: "))
            print("------------------------------------------------")
            if 1 <= seleccion <= len(puertos_disponibles):
                break
            else:
                print("Seleccion incorrecta. Intente de nuevo.")
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número.")

    puerto_seleccionado = puertos_disponibles[seleccion - 1]
    return puerto_seleccionado

puerto = seleccionar_puerto()
fecha = datetime.datetime.now().strftime("%d-%m-%Y")

vector = []
##########################################################################################################
def leer_entrada():
    global continuar
    while True:
        entrada = input("Presiona 'q' para salir: ").strip().lower()
        if entrada == 'q' or entrada == 'Q':
            continuar = False
            break

continuar = True

##########################################################################################################

def main():
    salir = False
    arduino = serial.Serial(puerto, 115200, timeout=1) #windows
    time.sleep(1) # Espera a que Arduino se inicialice
    arduino.flushInput()
    print("MENU PRINCIPAL")  
    print("Escriba el nombre del archivo a guardar (Por ejemplo: EXP1_Simon [Experimento_Autor])")
    nombre_archivo = input("Nombre del archivo:")
    print("Estas seguro?")
    print("1. Si")
    print("2. No")
    confirmar = input("Seleccione una opción (1 o 2): ")
    print("------------------------------------------------")
    if confirmar == '1':
        while not salir:
            print("1. Iniciar Calibracion")
            print("2. Iniciar Grabacion de datos")
            print("3. Salir")
            opcionn = input("Seleccione una opción (1 o 2): ")

            if opcionn == '1':
                print("------------------------------------------------")
                y1 = float(input("Seleccione el pH bajo"))
                instruccion = 'LEER\r'
                print("Enviando a Arduino: ", instruccion)
                time.sleep(1)
                arduino.write(instruccion.encode('utf-8'))
                x1 = arduino.readline().decode().strip() #7.01(y2) = 255(x2) // 4.01(y1) = 282(x1) // y=−0.1111111111111111x+35.343333333333334
                if x1 is not None and x1 != "":
                    try:
                        x1 = float(x1)
                    except ValueError:
                        print("El string no representa un número válido:", x1)
                else:
                    print("No se recibieron datos desde Arduino.")
                y2 = float(input("Seleccione el pH alto"))
                instruccion = 'LEER\r'
                print("Enviando a Arduino: ", instruccion)
                time.sleep(1)
                arduino.write(instruccion.encode('utf-8'))
                x2 = float(arduino.readline().decode().strip()) #7.01(y2) = 255(x2) // 4.01(y1) = 282(x1) // y=−0.1111111111111111x+35.343333333333334
                if x2 is not None and x2 != "":
                    try:
                        x2 = float(x2)
                    except ValueError:
                        print("El string no representa un número válido:", x1)
                else:
                    print("No se recibieron datos desde Arduino.")
                
                while True:
                    print("Quiere iniciar la calibracion")
                    print("1. Si")
                    print("2. No") 
                    iniciar_cal = input("Seleccione una opcion (1 o 2)")
                    if iniciar_cal =="1":
                        print("x1: ", x1)
                        print("x2: ", x2)
                        print("y1: ", y1)
                        print("y2: ", y2)
                        m = (y2 - y1) / (x2 - x1) 
                        b = y1 - m * x1
                        time.sleep(1)
                        print("m: ",m)
                        print("b: ",b)
                        time.sleep(1)  
                        break
                    if iniciar_cal =="2":
                        print("salir")
                        salir = True
            
            if opcionn == '2':
                print("------------------------------------------------")
                t_lectura = int(input("Seleccione un intervalo de tiempo (segs) entre lecturas:"))
                ruta_archivo = r'/home/manuelm/Documentos/DLAB/Registro-PH/Software/data/'+nombre_archivo+'_'+str(t_lectura)+'s_'+fecha+'.csv'
                while True:
                    print("Quiere iniciar el programa?")
                    print("1. Si")
                    print("2. No") 
                    iniciar = input("Seleccione una opcion (1 o 2)")
                    if iniciar =="1":
                        print("------------------------------------------------")
                        print("iniciando el programa...")
                        instruccion = 'LEER\r'
                        print("Enviando a Arduino: ", instruccion)
                        time.sleep(1)
                        arduino.write(instruccion.encode('utf-8'))
                        print("------------------------------------------------")
                        time.sleep(1)
                        print("Hora,data")
                        data_count = 0
                        thread_entrada = threading.Thread(target=leer_entrada)
                        thread_entrada.start()
                        try:
                            while True:
                                archivo_csv = open(ruta_archivo, mode='a', newline='')
                                escritor_csv = csv.writer(archivo_csv)
                                escritor_csv.writerow(["tiempo","data"])

                                try:
                                    while continuar:
                                        arduino.write(instruccion.encode('utf-8'))
                                        data = arduino.readline().decode().strip()
                                        if data is not None and data != "":
                                            data_count = data_count+1
                                            #['303', '305', '304', '304', '304', '305', '305', '304', '304', '304'] 1.68pH
                                            #['228', '229', '229', '229', '228', '230', '230', '229', '228', '230'] 10.01pH
                                            #y=−0.11106666666666666x+35.444266666666664
                                            data_pH = (m*float(data))+b #-(0.11106666666666666*float(data))+35.444266666666664
                                            data_pH = "{:.2f}".format(data_pH)
                                            tiempo_actual = time.strftime('%H:%M:%S') 
                                            archivo_csv = open(ruta_archivo, mode='a', newline='')
                                            escritor_csv = csv.writer(archivo_csv)
                                            escritor_csv.writerow([tiempo_actual,data_pH])
                                            archivo_csv.close()
                                            
                                            print(f"{tiempo_actual},{data_pH} pH\r", end="")
                                            time.sleep(t_lectura)
                                
                                except KeyboardInterrupt:
                                    print("Programa detenido por el usuario.")

                                thread_entrada.join()
                                break
                            print("------------------------------------------------")
                            time.sleep(1)
                            print("Saliendo, espere un momento...")
                            time.sleep(1)
                            print(".")
                            time.sleep(1)
                            print(".")
                            time.sleep(1)
                            print("Programa terminado")
                            time.sleep(2)
                            orden = 'R\r'
                            time.sleep(1)
                            arduino.write(orden.encode('utf-8'))
                            salir = True
                            break
    
    
                        except KeyboardInterrupt:
                            print("Programa detenido por el usuario.")
                                    
                                                   
                    elif iniciar =="2":
                        print("------------------------------------------------")
                        break
                    else:
                        print("Error")
    
    
            elif opcionn == '3':
                print("Terminado")
                arduino.close()
                time.sleep(3)
                salir = True
                
            else:
                print("Opción inválida. Por favor, selecciona una opción válida.")
    if confirmar == '2':
        print("Reiniciando...")
        time.sleep(3)
        main()
if __name__ == "__main__":
    main()

------------------------------------------------
SELECCIONE PUERTO SERIAL:
1. /dev/ttyACM0


Seleccione el número del puerto a usar:  1


------------------------------------------------
MENU PRINCIPAL
Escriba el nombre del archivo a guardar (Por ejemplo: EXP1_Simon [Experimento_Autor])


Nombre del archivo: qqq


Estas seguro?
1. Si
2. No


...

Quiere iniciar el programa?
1. Si
2. No


Seleccione una opcion (1 o 2) 1


------------------------------------------------
iniciando el programa...
Enviando a Arduino:  LEER
------------------------------------------------
Hora,data


Presiona 'q' para salir:  q


------------------------------------------------
Saliendo, espere un momento...
.
.
Programa terminado
